In [1]:
!pip install diffusers transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.9 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from urllib import request
import time
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

In [4]:
url = 'https://www.dogdrip.net/stock/category/291984713'
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text)

In [5]:
# 게시판 전체 태그 추출
whole_board = soup.find('table', class_ = 'ed table table-divider')
whole_board

<table class="ed table table-divider">
<thead>
<tr>
<th scope="col">번호</th> <th scope="col">제목</th> <th class="author ed" scope="col">글쓴이</th> <th scope="col">
				추천 수		      </th> <th scope="col">
		        날짜		      </th> <th scope="col">
		        조회 수		      </th> </tr>
</thead>
<tbody>
<tr class="notice">
<td class="ed notice text-small">
          공지        </td>
<td class="title">
<a class="ed link-reset" href="/stock/273533897?category=291984713&amp;page=1" style="">
<span class="ed title-link">
<span style="font-weight:bold">주식 / 재테크와 관련된 글을 올려주시기 바랍니다.</span> </span>
</a>
</td>
<td class="author" scope="row">
<a class="ed flex flex-left flex-middle link-reset member_13140068" href="#popup_menu_area" onclick="return false"><img alt="[레벨:50]" class="xe_point_level_icon" src="https://www.dogdrip.net/modules/point/icons/ddcoa/105.gif" style="vertical-align:middle;margin:0px 3px 2px 0px;" title="포인트:135317654드립력, 레벨:50/50"/>
		    overflow          </a>
</td>
<td class="ed voteNu

In [6]:
# whole_board 태그에서 각 게시물에 해당하는 태그 추출
all_title = whole_board.find_all('td', class_ = 'title')
all_title = [title.text for title in all_title]
all_title = [title.replace('\n', '').replace('[국장]', '').strip() for title in all_title]
all_title

['주식 / 재테크와 관련된 글을 올려주시기 바랍니다.',
 '에코프로 숏으로 달달하게 먹으신분들 다나와 매수 부탁드립...2',
 '와이지 점마 머고4',
 '개인적으로는 오늘 같이 에코프로 급락 나온게 좋아보임20',
 '에코프로 종토방 재밌네5',
 '이제 기업은행13',
 '난 에코프로 다르게 본다(원빈X)12',
 '국장 내일부터 진짜 ㅈ될예정2',
 '에코프로 20만원대 가야 조정 끝3',
 '에코프로 병신들ㅋㅋㅋ3',
 '에코프로 공매도 하차합니다^^732',
 '오늘 옵션만기일이라 하방 변동성 나온건가??3',
 '개쩌는거 보여줌33',
 '쯥 에코는 손절침6',
 '지금 에코후로 물린 센세들은1',
 '좆같은 국장2',
 '에코프로 전 회장 구속됐네1',
 '에코3',
 '어제 멘탈 바사삭해서 종가베팅한거6',
 '아침에 이트론 20번 물타면서 추매한결과..10',
 '주식을 발행하기 시작해서 그냥 많이 만들었습니다8']

In [7]:
# 각 게시글의 URL 추출
article_url = soup.find('table', class_ = 'ed table table-divider')
article_url_temp = article_url.find_all('a', class_ = 'ed link-reset')

In [8]:
all_article_url = []
for tag in article_url_temp : 
    all_article_url.append(tag['href'])
all_article_url

['/stock/273533897?category=291984713&page=1',
 '/stock/481445679?category=291984713&page=1',
 '/stock/481392650?category=291984713&page=1',
 '/stock/481385515?category=291984713&page=1',
 '/stock/481379749?category=291984713&page=1',
 '/stock/481379541?category=291984713&page=1',
 '/stock/481378841?category=291984713&page=1',
 '/stock/481379049?category=291984713&page=1',
 '/stock/481378828?category=291984713&page=1',
 '/stock/481376322?category=291984713&page=1',
 '/stock/481374652?category=291984713&page=1',
 '/stock/481374605?category=291984713&page=1',
 '/stock/481373102?category=291984713&page=1',
 '/stock/481372887?category=291984713&page=1',
 '/stock/481372870?category=291984713&page=1',
 '/stock/481372541?category=291984713&page=1',
 '/stock/481371228?category=291984713&page=1',
 '/stock/481370627?category=291984713&page=1',
 '/stock/481367747?category=291984713&page=1',
 '/stock/481365714?category=291984713&page=1',
 '/stock/481224820?category=291984713&page=1']

In [9]:
all_article_url = [('https://www.dogdrip.net' + url) for url in all_article_url]
all_article_url

['https://www.dogdrip.net/stock/273533897?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481445679?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481392650?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481385515?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481379749?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481379541?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481378841?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481379049?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481378828?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481376322?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481374652?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481374605?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481373102?category=291984713&page=1',
 'https://www.dogdrip.net/stock/481372887?category=291984713&page=1',
 'https://www.dogdri

In [10]:
# 게시글의 내용을 크롤링
url = 'https://www.dogdrip.net/stock/475288748?category=291984713&page=1'
article_id = url.split('?category=')[0]
article_id = article_id.split('https://www.dogdrip.net/stock/')[1]

res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text)

In [11]:
soup.find_all('div', class_ = 'document_' + article_id + '_0 rhymix_content xe_content')

[<div class="document_475288748_0 rhymix_content xe_content"><p>이제 빼야한다는게 너무 마음아프다.. 그래도 50프로 먹고 나왔으니 기분은 좋다</p>
 <p> </p>
 <p>남은 사람들은 계속 추매해서 집 사자! </p>
 <p> </p>
 <p>좀만 버티면 평수가 달라진다</p></div>]

In [12]:
# 스크래핑한 데이터를 정리
df = pd.DataFrame({'title' : all_title, 'URL' : all_article_url})
df


,title,URL
0,주식 / 재테크와 관련된 글을 올려주시기 바랍니다.,https://www.dogdrip.net/stock/273533897?catego...
1,에코프로 숏으로 달달하게 먹으신분들 다나와 매수 부탁드립...2,https://www.dogdrip.net/stock/481445679?catego...
2,와이지 점마 머고4,https://www.dogdrip.net/stock/481392650?catego...
3,개인적으로는 오늘 같이 에코프로 급락 나온게 좋아보임20,https://www.dogdrip.net/stock/481385515?catego...
4,에코프로 종토방 재밌네5,https://www.dogdrip.net/stock/481379749?catego...
5,이제 기업은행13,https://www.dogdrip.net/stock/481379541?catego...
6,난 에코프로 다르게 본다(원빈X)12,https://www.dogdrip.net/stock/481378841?catego...
7,국장 내일부터 진짜 ㅈ될예정2,https://www.dogdrip.net/stock/481379049?catego...
8,에코프로 20만원대 가야 조정 끝3,https://www.dogdrip.net/stock/481378828?catego...
9,에코프로 병신들ㅋㅋㅋ3,https://www.dogdrip.net/stock/481376322?catego...


In [13]:
batch_inputs = df['title'].values.tolist()

In [14]:
BASE_URL = "https://gall.dcinside.com/mgallery/board/lists/"
ARTICLE_BASE_URL = "https://gall.dcinside.com"

# 헤더 설정
headers = [
{
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
},
]
res_list = []
#몇 페이지부터 몇 페이지까지
for i in range(1, 60):
    # 파라미터 설정
    params = {'id': 'kospi','page':i}

    response = requests.get(BASE_URL, params=params, headers=headers[0])

    soup = BeautifulSoup(response.content, 'html.parser')

    #실질적 글 목록 부분
    article_list = soup.find('tbody').find_all('tr')

    # 한 페이지에 있는 모든 게시물을 긁어오는 코드 
    for tr_item in article_list:    
        # # 이미지가 있는 게시물일 경우만 탐색 시작 start
        # image_flag = tr_item.find('em', class_='icon_img icon_pic')

        # if image_flag is None:
        #     continue
        # # 이미지가 있는 게시물일 경우만 탐색 시작 end

        # print('+'*12)

        # 제목 추출
        title_tag = tr_item.find('a', href=True)
        title = title_tag.text

        print("T: ", title)
        # print("주소: ", title_tag['href'])
        res_list.append(title)

        # # 이미지가 있는 게시물에 request
        # article_response = requests.get(ARTICLE_BASE_URL + title_tag['href'], headers=headers[0])
        # # print("url: ", article_response.url)

        # article_id = (title_tag['href'].split('no=')[1]).split('&')[0]
        # # print("게시물 ID : ", article_id)

        # article_soup = BeautifulSoup(article_response.content, 'html.parser')

        # # 게시물 부분의 태그
        # article_contents = article_soup.find('div', class_='writing_view_box').find_all('div')


T:  어떻게든 더 성공하고 싶은 야망이 보이는 스타는?
T:  [해외선물] 빠른주문화면 외않써?
T:  [이베스트] 마이크로나스닥 수수료 무료 이벤트~
T:  주역은 피파 4와 던파, 넥슨 1분기 영업이익 46% 상승
T:  인생성공3원칙ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
T:  [공지] 코스피 마이너 갤러리 공지 모음
T:  코스피갤 공식 투자입문섴ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
T:  매매 너무어렵다ㄷㄷㄷㄷㄷㄷ
T:  윤석열 관련주 “NE능률“ 근황을.あらぼざ
T:  야 이 씹덕 새끼들아
T:  넥슨 자체는 괜찮은 주식이라고 생각함
T:  목표 수익률이란건 어려운거야
T:  남국 이새끼 존나 웃기네 ㅋㅋㅋㅋㅋㅋㅋ
T:  군대 문제랑 노인 일자리 동시에 해결하는 방법있네
T:  님들이라면 100프로 벌었다가
T:  너네 보통 보유기간하고 목표수익률이 몇이냐?
T:  마이크로소프트 금여인상 보류
T:  금양이 작전주의 정석이네
T:  ???점마 좆터리?컄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
T:  넥슨 내일 설거지같은데
T:  내일 에코프로로 천하제일 단타대회 열릴듯
T:  경기침체는 필수자노?컄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
T:  맨날 깡통전세도 다 전세사기 ㅇㅈㄹ하니까 피해자도 잘못 이러는 거지
T:  내일 쏘는 차트 하나 보여준다
T:  vip빤스런 했자노?컄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
T:  전세사기는 본인잘못임. 
T:  에코호로 콜옵션 사고 싶다
T:  캐나다 상시통화스왑 걸려있는 진짜 형제국 아니냐
T:  생각에 잠길때마다
T:  오 빗코 본전왔따
T:  내일이 이번주 마지막 도박장이네
T:  개잡주 매집봉 매매 가르쳐줌
T:  주식하는 사람이 품격을 논하다니
T:  와 소름 돋는 반전이네
T:  품격높은 주식커뮤 특
T:  님들 무섭게 왜그럼
T:  이마트 트레이더스 입장하는 법
T:  여동생도 삼일한 해라
T:  미장이랑 조선장이랑 마주할 때 느낌이 다르노 . . . . . . . .
T:  

In [15]:
batch_inputs.extend(res_list[7:])

print(batch_inputs[1:])
print(len(batch_inputs[1:]))

k_vix_data = batch_inputs[1:]

['에코프로 숏으로 달달하게 먹으신분들 다나와 매수 부탁드립...2', '와이지 점마 머고4', '개인적으로는 오늘 같이 에코프로 급락 나온게 좋아보임20', '에코프로 종토방 재밌네5', '이제 기업은행13', '난 에코프로 다르게 본다(원빈X)12', '국장 내일부터 진짜 ㅈ될예정2', '에코프로 20만원대 가야 조정 끝3', '에코프로 병신들ㅋㅋㅋ3', '에코프로 공매도 하차합니다^^732', '오늘 옵션만기일이라 하방 변동성 나온건가??3', '개쩌는거 보여줌33', '쯥 에코는 손절침6', '지금 에코후로 물린 센세들은1', '좆같은 국장2', '에코프로 전 회장 구속됐네1', '에코3', '어제 멘탈 바사삭해서 종가베팅한거6', '아침에 이트론 20번 물타면서 추매한결과..10', '주식을 발행하기 시작해서 그냥 많이 만들었습니다8', '매매 너무어렵다ㄷㄷㄷㄷㄷㄷ', '윤석열 관련주 “NE능률“ 근황을.あらぼざ', '야 이 씹덕 새끼들아', '넥슨 자체는 괜찮은 주식이라고 생각함', '목표 수익률이란건 어려운거야', '남국 이새끼 존나 웃기네 ㅋㅋㅋㅋㅋㅋㅋ', '군대 문제랑 노인 일자리 동시에 해결하는 방법있네', '님들이라면 100프로 벌었다가', '너네 보통 보유기간하고 목표수익률이 몇이냐?', '마이크로소프트 금여인상 보류', '금양이 작전주의 정석이네', '???점마 좆터리?컄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '넥슨 내일 설거지같은데', '내일 에코프로로 천하제일 단타대회 열릴듯', '경기침체는 필수자노?컄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '맨날 깡통전세도 다 전세사기 ㅇㅈㄹ하니까 피해자도 잘못 이러는 거지', '내일 쏘는 차트 하나 보여준다', 'vip빤스런 했자노?컄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '전세사기는 본인잘못임. ', '에코호로 콜옵션 사고 싶다', '캐나다 상시통화스왑 걸려있는 진짜 형제국 아니냐', '생각에 잠길때마다', '오 빗코 본전왔따', '내일이 이번주 마지막 도박장이네', '개잡주 매집봉 매매

In [16]:
class k_vix_dataloader(Dataset):
    def __init__(self, data_list):
        self.data = data_list
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # 인덱스에 해당하는 데이터 반환
        return self.data[index]

In [17]:
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

In [18]:
dataset = k_vix_dataloader(k_vix_data)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [19]:
get_results = []
for data in dataloader:
    print(data)
    inputs = tokenizer(data, return_tensors="pt", padding=True, truncation=False)
    outputs = model(**inputs)
    get_results.extend(outputs.logits)
    

['분봉하나에 20포짜린데  존나 작아보이노 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '조선주는 어렵다.', 'mts 나무가 좋아보이긴하는데 토스못버리겠다', '코스닥 무너지던 말던', '파벳 넌 노가다충이니간 당연히 시키는대로 해여지 ㅡ ㅡ', '긴난국 계좌 영장 기각 판사, 라임 김봉현 영장도 기각했었다(1보)', '낼부터 에ㅈ호로 하한가 직행~ ', '에코프로 전 회장 구속됐네1']
['1억도없는 거지를거지라부르는게 죄냐고 씨발아ㅋ', '참 요즘 인터넷보면 아픈사람 많이보인다', '전두환때 실업률', '하닉 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '에코프로 드디어 뒤지는구나', '난 중국 저가 공세는 이제 10년 안남았다고 봄 ㅇㅇ', '병걸리면그냥뒤져이개새끼얔ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '이씨 삼형제 여따 약팔아서 파벗한테 욕먹나보네ㅋ']
['외쳐 코스닥 600 코스피 2100 ㅋㅋㅋㅋㅋ', '입답치라고 ㅇㅅㄲ야 ㅡㅡ 대지고 십냐? 고통 맛 보여줘??', '코스닥 나스닥 따라갈거 같다', '지금 이마트 잡으면 골로가냐?', '셀인메이 스타또~~', '니애미모빌 10만주를 던지노 ㅡㅡ', '저런식이면 전망이라는게 결국 끼워 맞추기 아닌가?', '테라사이언스 ㅈ된애들']
['에코프로 내일 상한가 간다', '한미글로벌 -20% 처맞았는데 어떡야 되냐?', '공구우먼 당일 20%운지 맞췃는데?ㅋ', '쉬멜이나 여장남자한테 박혀본적 있으면 개추', '미주갤 분단됐네 ㅇㅅㅇ', '오늘 장예상해준다 반도체고 나발이고', '큰돈 힌번에 복구하고싶으면 옵션해라 ', '2010년에 JYP 1억치샀으면 지금까지 100억 먹은거냐?']
['이모지 왜 못 쓰냐 ', '돈깡 개고수였네.. 애코호로 숏 쳤다며 ㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷㄷ', '쾌속운지를 알고 싶으면 주식을 해라 ', '가스가 없으면 베스핀 가스를 캐면 되잖아', '경기침체라는것도 좀 어거지로 만든 프레임같긴 함', '재미로하니까주식이잘되네  ', '에코프로 고점은 10년 지나도 다시 올지 장담 못한다', '의외로 수익좋은 

In [20]:
all_pred = torch.stack(get_results, dim=0)

act = nn.Softmax(dim=1)
pred = act(all_pred)

In [21]:
res = torch.mean(pred, 0).tolist() # neg, neu, pos
print(res)

[0.08740676939487457, 0.8713562488555908, 0.0412370041012764]


In [22]:
neg = res[0] / (res[0] + res[2])
pos = res[2] / (res[0] + res[2]) 

print(neg, pos)

0.6794481148944981 0.3205518851055019
